In [257]:
import os
import xml.etree.ElementTree as ET
import json
import re
import datetime

In [258]:
gpx_file_names = [x for x in os.listdir('../assets/gps_tracks') if x.endswith('gpx')]
#gpx_file_names = ['d10_01_ride.gpx']
gpx_files = [os.path.join('../assets/gps_tracks', x) for x in gpx_file_names]
gpx_file_names[0]

'd01_01_ride.gpx'

In [259]:
all_coordinates = []
properties = []
for gpx_file, gpx_file_name in zip(gpx_files, gpx_file_names):
    parts = gpx_file_name.split('_')
    day = int(parts[0][1:])
    exercise_type = parts[2].split('.')[0]
    
    
    tree = ET.parse(gpx_file)
    root = tree.getroot()

    trk = [c for c in root if c.tag.endswith('trk')]
    trksegs = [t for t in trk[0] if t.tag.endswith('trkseg')]
    trkpts = []

    for trkseg in trksegs:
        trkpts.extend([p for p in trkseg if p.tag.endswith('trkpt')])
    coordinates = [(float(t.attrib['lon']), float(t.attrib['lat'])) for t in trkpts]
    # times = [datetime.datetime.strptime([t for t in x if t.tag.endswith('time')][0].text, "%Y-%m-%dT%H:%M:%SZ") for x in trkpts]
    last_idx = 0
    for c0, c1, i in zip(coordinates[:-1], coordinates[1:], range(len(coordinates)-1)):
        d2 = (c1[1] - c0[1])**2 + (c1[0] - c0[0])**2
        if d2 > 0.03**2:
            all_coordinates.append(coordinates[last_idx:i+1])
            last_idx = i+1
            properties.append({
                "day": day,
                "category": exercise_type
            })
    
    if last_idx < len(coordinates):
        all_coordinates.append(coordinates[last_idx:])

        properties.append({
            "day": day,
            "category": exercise_type
        })


In [260]:
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": c
            },
            "properties": p
        } for c,p in zip(all_coordinates, properties)]
}
geojson_string = json.dumps(geojson)

In [261]:
matches = re.findall(r'"-?\d+.\d+"', geojson_string)

new_string = re.sub(r'"(-?\d+.\d+)"', lambda x: x.group(1), geojson_string)
new_string[:50000]

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "LineString", "coordinates": [[-1.121485, 52.37534], [-1.12135, 52.375283], [-1.121242, 52.375179], [-1.121699, 52.374698], [-1.121975, 52.374476], [-1.122574, 52.37406], [-1.122717, 52.373991], [-1.122841, 52.374], [-1.122974, 52.374048], [-1.123733, 52.374372], [-1.124551, 52.374709], [-1.124903, 52.374847], [-1.125696, 52.375096], [-1.125952, 52.375169], [-1.126611, 52.375347], [-1.127479, 52.375538], [-1.127758, 52.375568], [-1.127845, 52.375585], [-1.128019, 52.375631], [-1.12857, 52.375922], [-1.128654, 52.376006], [-1.128709, 52.376095], [-1.128761, 52.376286], [-1.128761, 52.376286], [-1.128761, 52.376286], [-1.129033, 52.37838], [-1.129179, 52.378598], [-1.129325, 52.378816], [-1.129618, 52.379251], [-1.129643, 52.379363], [-1.129669, 52.379474], [-1.129694, 52.379586], [-1.12972, 52.379697], [-1.129874, 52.380365], [-1.130051, 52.380477], [-1.13028, 52.380515], [-1.130508, 52.380554], [-1.130

In [262]:
with open('../assets/gps_tracks/all_tracks.json', 'w') as f:
    f.write(new_string)
